##Projeto final do bootcamp de engenheiro de bigdata

In [0]:
import urllib 
urllib.request.urlretrieve("https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar", "/tmp/covid.rar")

Out[39]: ('/tmp/covid.rar', <http.client.HTTPMessage at 0x7fb125805760>)

In [0]:
%sh
sudo apt install --assume-yes unrar

WARNING: apt does not have a stable CLI interface. Use with caution in scripts.

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
 unrar
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 113 kB of archives.
After this operation, 406 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 unrar amd64 1:5.6.6-2build1 [113 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 113 kB in 0s (241 kB/s)
Selecting previously unselected package unrar.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 95834 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a5.6.6-2build1_amd64.deb ...
Unpacking unrar (1:5.6.6-2build1) ...
Setting up unrar (1:5.6.6-2build1) ...
update-alternatives: using /usr/bin/unrar-nonfree to provide /usr/bin/unrar (unrar) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/unrar.1.gz because associated file /usr/share/man/man1/unrar-nonfree.1.gz (of link group unrar) doesn't exist
Processing triggers for man-db (2.9.1-1) ...

In [0]:
%sh
ls -la /tmp/covid*
unrar x /tmp/covid_excel.rar


-rw-r--r-- 1 root root 12445782 Aug 10 01:20 /tmp/covid.rar
-rw-r--r-- 1 root root 12445782 Aug 10 00:26 /tmp/covid_excel.rar
-rw-r--r-- 1 root root 12445782 Aug 10 00:20 /tmp/covid_excel.zip

UNRAR 5.61 beta 1 freeware Copyright (c) 1993-2018 Alexander Roshal


Extracting from /tmp/covid_excel.rar

Extracting HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv  21% OK 
Extracting HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv  52% OK 
Extracting HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv  86% 97% OK 
Extracting HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv  99% OK 
All OK

In [0]:
#mover temp file para DBFS
dbutils.fs.mv("file:/databricks/driver/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv", "dbfs:/tmp/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv")  
dbutils.fs.mv("file:/databricks/driver/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv", "dbfs:/tmp/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv")  
dbutils.fs.mv("file:/databricks/driver/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv", "dbfs:/tmp/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv")  
dbutils.fs.mv("file:/databricks/driver/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv", "dbfs:/tmp/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv")  

Out[46]: True

In [0]:
%fs
ls /tmp

path,name,size,modificationTime
dbfs:/tmp/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv,HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv,62492959,1660095125000
dbfs:/tmp/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv,HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv,76520681,1660095126000
dbfs:/tmp/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv,HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv,91120916,1660095127000
dbfs:/tmp/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv,HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv,3046774,1660095127000


In [0]:
import os
import sys
import json
import pyspark.sql.functions as f
from pyspark.sql.functions import when, col,round, sum
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, DecimalType
from pyspark import SparkContext, SparkConf, SQLContext, HiveContext
from collections import namedtuple

In [0]:
schema = StructType([ \
    StructField("regiao",StringType(),True), \
    StructField("estado",StringType(),True), \
    StructField("municipio",StringType(),True), \
    StructField("coduf", IntegerType(), True), \
    StructField("codmun", IntegerType(), True), \
    StructField("codRegiaoSaude", IntegerType(), True), \
    StructField("nomeRegiaoSaude",StringType(),True), \
    StructField("data",TimestampType(),True), \
    StructField("semanaEpi",IntegerType(),True), \
    StructField("populacaoTCU2019", IntegerType(), True), \
    StructField("casosAcumulado", DecimalType(), True), \
    StructField("casosNovos", IntegerType(), True) ,\
    StructField("obitosAcumulado", IntegerType(), True), \
    StructField("obitosNovos", IntegerType(), True), \
    StructField("Recuperadosnovos", IntegerType(), True), \
    StructField("emAcompanhamentoNovos", IntegerType(), True), \
    StructField("interior/metropolitana", IntegerType(), True), \
  ])

In [0]:
df = spark.read\
 .option("header","true")\
 .option("schema",schema)\
 .option("mode","DROPMALFORMED")\
 .option("delimiter", ";")\
 .option("inferSchema", "true")\
 .csv("dbfs:/tmp/*.csv")
display(df)

regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
Brasil,null,null,76,null,null,null,2020-02-25T00:00:00.000+0000,9,210147125,0.0,0,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-02-26T00:00:00.000+0000,9,210147125,1.0,1,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-02-27T00:00:00.000+0000,9,210147125,1.0,0,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-02-28T00:00:00.000+0000,9,210147125,1.0,0,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-02-29T00:00:00.000+0000,9,210147125,2.0,1,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-03-01T00:00:00.000+0000,10,210147125,2.0,0,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-03-02T00:00:00.000+0000,10,210147125,2.0,0,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-03-03T00:00:00.000+0000,10,210147125,2.0,0,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-03-04T00:00:00.000+0000,10,210147125,3.0,1,0,0,null,null,null
Brasil,null,null,76,null,null,null,2020-03-05T00:00:00.000+0000,10,210147125,7.0,4,0,0,null,null,null


In [0]:
df.count()

Out[56]: 2624943

In [0]:
df.printSchema()

root
-- regiao: string (nullable = true)
-- estado: string (nullable = true)
-- municipio: string (nullable = true)
-- coduf: integer (nullable = true)
-- codmun: integer (nullable = true)
-- codRegiaoSaude: integer (nullable = true)
-- nomeRegiaoSaude: string (nullable = true)
-- data: timestamp (nullable = true)
-- semanaEpi: integer (nullable = true)
-- populacaoTCU2019: integer (nullable = true)
-- casosAcumulado: double (nullable = true)
-- casosNovos: integer (nullable = true)
-- obitosAcumulado: integer (nullable = true)
-- obitosNovos: integer (nullable = true)
-- Recuperadosnovos: integer (nullable = true)
-- emAcompanhamentoNovos: integer (nullable = true)
-- interior/metropolitana: integer (nullable = true)

In [0]:
df= df.withColumnRenamed("codRegiaoSaude","codregiaosaude")\
	.withColumnRenamed("nomeRegiaoSaude","nomeregiaosaude")\
	.withColumnRenamed("semanaEpi","semanaepi")\
	.withColumnRenamed("populacaoTCU2019","populacaotcu2019")\
	.withColumnRenamed("casosAcumulado","casosacumulado")\
	.withColumnRenamed("casosNovos","casosnovos")\
	.withColumnRenamed("obitosAcumulado","obitosacumulado")\
	.withColumnRenamed("obitosNovos","obitosnovos")\
	.withColumnRenamed("emAcompanhamentoNovos","emacompanhamentonovos")\
	.withColumnRenamed("interior/metropolitana","interiormetropolitana")\
    .withColumnRenamed("Recuperadosnovos","recuperadosnovos")

In [0]:
df.write.mode("overwrite").partitionBy("municipio").saveAsTable("covid")

In [0]:
df_covid = hc.read.table("covid")

In [0]:
df_covid.withColumn('recuperadosnovos',when(col('recuperadosnovos').isNull(),0)\
                    .otherwise(col('recuperadosnovos')))\
                    .agg(sum(col("recuperadosnovos"))).show(20)

+---------------------+
sum(recuperadosnovos)|
+---------------------+
 2920055795|
+---------------------+

In [0]:
df_covid.select(col("municipio")).where(col("municipio").isNotNull()).show(3)

+---------------+
 municipio|
+---------------+
Abadia de Goiás|
Abadia de Goiás|
Abadia de Goiás|
+---------------+
only showing top 3 rows

In [0]:
df_brazil = df_covid.where(col("regiao")=='Brasil')\
               .orderBy(col("data").desc())\
               .limit(1)

In [0]:
df1 = df_brazil\
           .select(col('recuperadosnovos'),col('emacompanhamentonovos'))\
           .limit(1) 
df1.show()

+----------------+---------------------+
recuperadosnovos|emacompanhamentonovos|
+----------------+---------------------+
 17262646| 1065477|
+----------------+---------------------+

In [0]:
df2 = df_brazil\
           .select(col('casosnovos'),col('casosacumulado'))\
           .limit(1) 
df2.show()  

+----------+--------------+
casosnovos|casosacumulado|
+----------+--------------+
 62504| 1.8855015E7|
+----------+--------------+

In [0]:
df3 = df_brazil\
           .withColumn("letalidade",col("obitosacumulado")/col('casosacumulado')*100)\
           .withColumn("letalidade",round(col("letalidade"),2))\
           .withColumn("mortalidade",col("obitosacumulado")/col('populacaotcu2019')*100000)\
           .withColumn("mortalidade",round(col("mortalidade"),2))\
           .select(col('obitosacumulado'),col('obitosnovos'),col("letalidade"),col("mortalidade"))\
           .limit(1)
df3.show()

+---------------+-----------+----------+-----------+
obitosacumulado|obitosnovos|letalidade|mortalidade|
+---------------+-----------+----------+-----------+
 526892| 1780| 2.79| 250.73|
+---------------+-----------+----------+-----------+

In [0]:
from pyspark.sql.functions import lit
df_covid.withColumn("country",lit("Brazil")).display()

regiao,estado,municipio,coduf,codmun,codregiaosaude,nomeregiaosaude,data,semanaepi,populacaotcu2019,casosacumulado,casosnovos,obitosacumulado,obitosnovos,recuperadosnovos,emacompanhamentonovos,interiormetropolitana,country
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-01T00:00:00.000+0000,53,7457,99.0,0,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-02T00:00:00.000+0000,53,7457,99.0,0,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-03T00:00:00.000+0000,1,7457,109.0,10,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-04T00:00:00.000+0000,1,7457,110.0,1,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-05T00:00:00.000+0000,1,7457,118.0,8,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-06T00:00:00.000+0000,1,7457,119.0,1,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-07T00:00:00.000+0000,1,7457,126.0,7,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-08T00:00:00.000+0000,1,7457,126.0,0,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-09T00:00:00.000+0000,1,7457,140.0,14,1,0,null,null,0,Brazil
Sul,PR,Abatiá,41,410010,41018,18ª RS CORNELIO PROCOPIO,2021-01-10T00:00:00.000+0000,2,7457,140.0,0,1,0,null,null,0,Brazil
